# 📘 Nuggetizer: A lightweight nugget-based evaluation framework for pyterrier-rag

## 📌 Introduction
- Objective: Demonstrate how to use NuggetizerRAG, a personal library for nugget-based evaluation in Retrieval-Augmented Generation (RAG).
- Context: Inspired by the AutoNuggetizer framework from the TREC 2024 RAG Track.
- Use case: Provide interpretable and automatic evaluation metrics for open-domain QA and generation tasks using PyTerrier pipelines.

## 🎯 Motivation and Background
- The Problem: Traditional RAG evaluations rely on lexical overlap or ROUGE scores, which miss semantic correctness.
- The Solution: Nugget evaluation, originally proposed in TREC QA 2003, revived by AutoNuggetizer, uses semantically atomic facts (“nuggets”) to evaluate answers.
- Inspiration: This library reimplements a simplified, local version of AutoNuggetizer with modular hooks into PyTerrier and HuggingFace models.

## ⚙️ Installation and Setup

In [1]:
!pip install git+https://github.com/MattiWe/ir_datasets.git@add-msmarco-v2.1-trec-rag

  Cloning https://github.com/MattiWe/ir_datasets.git (to revision add-msmarco-v2.1-trec-rag) to /tmp/pip-req-build-d6j4viou
  Running command git clone --filter=blob:none --quiet https://github.com/MattiWe/ir_datasets.git /tmp/pip-req-build-d6j4viou
  Running command git checkout -b add-msmarco-v2.1-trec-rag --track origin/add-msmarco-v2.1-trec-rag
  Switched to a new branch 'add-msmarco-v2.1-trec-rag'
  Branch 'add-msmarco-v2.1-trec-rag' set up to track remote branch 'add-msmarco-v2.1-trec-rag' from 'origin'.
  Resolved https://github.com/MattiWe/ir_datasets.git to commit bd018b783e3d25942b69290f7be19eeb929022c2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip install -q python-terrier pyterrier_t5 pyterrier_pisa

In [3]:
!pip install -q git+https://github.com/terrierteam/pyterrier_rag.git

In [4]:
!pip install -q --no-deps ../.

In [5]:
import pyterrier as pt
from pyterrier_rag.backend import Backend

/opt/miniconda3/envs/nuggetizer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [6]:
import ir_datasets
dataset = ir_datasets.load('msmarco-segment-v2.1')

In [7]:
pt_dataset = pt.get_dataset("irds:msmarco-segment-v2.1")

# Pipelines

In [8]:
from pyterrier_pisa import PisaIndex
from pyterrier_t5 import MonoT5ReRanker

def rename_segment(run):
    run = run.rename(columns={"segment": "text"})
    return run
rename_pipe = pt.apply.generic(rename_segment)

index = PisaIndex('/mnt/indices/msmarco-segment-v2.1.pisa/')
bm25_ret = index.bm25() >> pt.text.get_text(pt_dataset, "segment") >> rename_pipe
monoT5 = MonoT5ReRanker(batch_size=64, verbose=False)
monoT5_ret = bm25_ret % 10 >> monoT5

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


# Building a baseline retrieval run to generate baseline nuggets

In [9]:
import pandas as pd
df = pd.read_csv("../../diversification/diversy-rag/datasets/TREC-RAGgy/raggy-dev.tsv", sep="\t", dtype=str)
topics_df  = df[["qid", "query"]]
answers_df = df[["qid", "query", "gold_answer"]]

baseline = (monoT5_ret)(topics_df.head(10))
baseline

# Nuggetizer setup

In [10]:
from pyterrier_rag.backend import HuggingFaceBackend

backend =  HuggingFaceBackend("hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4",
                                          max_new_tokens=2048,
                                          model_args={
                                              "device_map": "cuda"
                                          }
                                         )

Loading checkpoint shards: 100%|██████████| 9/9 [00:05<00:00,  1.72it/s]


In [11]:
from fastchat.conversation import register_conv_template, get_conv_template, Conversation, SeparatorStyle

register_conv_template(
    Conversation(
        name="meta-llama-3.1-sp",
        system_message="",
        roles=("user", "assistant"),
        sep_style=SeparatorStyle.ADD_COLON_SINGLE,
        sep="\n",
        messages=[],
    )
)

conv_template = get_conv_template("meta-llama-3.1-sp")

In [12]:
import pandas as pd

def save_csv(path, content):
    content.to_csv(path, index=False)

def load_csv(path):
    try:
        content = pd.read_csv(path)
        return content
    except Exception:
        return None

In [13]:
from open_nuggetizer.nuggetizer import Nuggetizer

nuggetizer = Nuggetizer(
    backend=backend, 
    conversation_template=conv_template,
    verbose=True
)

nuggets = load_csv("nuggets.csv")
if nuggets is None:
    nuggets = nuggetizer.create(baseline)
    save_csv("nuggets.csv", nuggets)

scored_nuggets = load_csv("scored_nuggets")
if scored_nuggets is None:
    scored_nuggets = nuggetizer.score(nuggets)
    save_csv("scored_nuggets.csv", scored_nuggets)

pt.apply.by_query():   0%|          | 0/225 [00:00<?, ?it/s]

LEN:12 - 10



  0%|          | 0/2 [00:00<?, ?window/s]/opt/miniconda3/envs/nuggetizer/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


PROMPT:
 ['You are NuggetizeScoreLLM, an intelligent assistant that can label a list of atomic nuggets based on their importance for a given search query.\nuser: Based on the query, label each of the 2 nuggets either a vital or okay based on the following criteria. Vital nuggets represent concepts that must be present in a “good” answer; on the other hand, okay nuggets contribute worthwhile information about the target but are not essential. Return the list of labels in a Pythonic list format (type: List[str]). The list should be in the same order as the input nuggets. Make sure to provide a label for each nugget.\n\nSearch Query: are landlords liable if someone breaks in a hurts tenant\nNugget List: [\'Landlord not liable for conditions arising after tenant takes possession\', "Landlord can be held liable for tenant\'s behavior if aware and does nothing"]\n\nOnly return the list of labels (List[str]). Do not explain.\nLabels:\n']


From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.

 50%|█████     | 1/2 [00:03<00:03,  3.18s/window]

...

 33%|███▎      | 1/3 [00:02<00:05,  2.91s/window]

PROMPT:
 ['You are NuggetizeScoreLLM, an intelligent assistant that can label a list of atomic nuggets based on their importance for a given search query.\nuser: Based on the query, label each of the 10 nuggets either a vital or okay based on the following criteria. Vital nuggets represent concepts that must be present in a “good” answer; on the other hand, okay nuggets contribute worthwhile information about the target but are not essential. Return the list of labels in a Pythonic list format (type: List[str]). The list should be in the same order as the input nuggets. Make sure to provide a label for each nugget.\n\nSearch Query: information about who howard gardner and what does he do\nNugget List: ["Gardner\'s theory of multiple intelligences includes eight different types of intelligences", \'Linguistic-Verbal intelligence\', \'Logical-Mathematical intelligence\', \'Visual-Spatial intelligence\', \'Bodily-Kinesthetic intelligence\', \'Musical-Rhythmic intelligence\', \'Interperson

 67%|██████▋   | 2/3 [00:09<00:05,  5.26s/window]

PROMPT:
 ["You are NuggetizeScoreLLM, an intelligent assistant that can label a list of atomic nuggets based on their importance for a given search query.\nuser: Based on the query, label each of the 10 nuggets either a vital or okay based on the following criteria. Vital nuggets represent concepts that must be present in a “good” answer; on the other hand, okay nuggets contribute worthwhile information about the target but are not essential. Return the list of labels in a Pythonic list format (type: List[str]). The list should be in the same order as the input nuggets. Make sure to provide a label for each nugget.\n\nSearch Query: information about who howard gardner and what does he do\nNugget List: ['Howard Gardner is an American psychologist', 'He specializes in cognitive and developmental psychology', 'He is best known for his theory of multiple intelligences', 'Gardner believes that the way people usually think about intelligence is too narrow', 'He was born on July 11, 1943 in S

100%|██████████| 3/3 [00:17<00:00,  5.92s/window]


# Evaluation

In [14]:
from pyterrier_rag.prompt import Concatenator
from pyterrier_rag.readers import Reader

reader = Reader(backend)
rag_pipeline = monoT5_ret % 3 >> Concatenator() >> reader

(monoT5_ret % 3 >> Concatenator() >> reader)(topics_df.head(1))

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/opt/miniconda3/envs/nuggetizer/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


,prompt,qid,query_0,qanswer
0,A chat between a curious human and an artifici...,23287,are landlords liable if someone breaks in a hu...,Landlords are generally not liable for injuri...


In [15]:
import pyterrier_rag.measures

results = pt.Experiment(
    [
        rag_pipeline
    ],
    topics_df.head(2), 
    answers_df,
    [pyterrier_rag.measures.F1, nuggetizer.VitalScore()],
    #batch_size=25,
    names=['baseline retriever'],
)

/opt/miniconda3/envs/nuggetizer/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


KeyError: 'qid'